# *입력되는 엑셀의 기본 조건

## 1. 제목 행은 총 4행 > 템플릿 파일들 기준으로 맞추자!
- 1행 : 2023년_ [4-마. 재적 학생 현황_] 같은 설명
- 2~3행 : 빈행 (체크 필요)
- 4행 ~ 7행 : 제목행 (셀들은 병합되어있으며 변경됨)

## 2. 삭제되는 공통열은 university 테이블 참조
- '설립구분'
- '학교종류'
- '지역'
- '상태'

## 3. 엑셀의 컬럼명과 일치하는 컬럼명으로 자동 매칭되도록...

In [1]:
import os, sys
# 판다스
#!pip install pandas
#%pip install pandas

# Pandas
#%pip install pandas
import pandas as pd


import source.schema as sch
# print(sch.table_dic['contract_department'])
# sys.exit(0)


#################################################
# 디버그 모드 설정
#################################################
def _dbg_(a='', b='', c='', d='', e=''):
    print('[dbg]', '(', sys._getframe(1).f_code.co_name, ') ', a, b, c, d, e)

In [2]:
#################################################
# DB 접속 설정
#################################################

#%pip install pymysql
import pymysql

def db_setting(ip, id, pwd, dbname):
    conn = pymysql.connect(host=ip, user=id, password=pwd, db=dbname)
    curs = conn.cursor(pymysql.cursors.DictCursor)

    if conn == None:
        print('DB 연결 오류')
    elif curs == None:
        print('curs 연결 오류')
        
    return conn, curs



In [3]:
#################################################
# 물리적 파일명 검증, 각 디렉토리에서 일치하는 파일명 기준으로 일괄 작업 수행
#################################################

#디렉토리의 전체 파일명 읽기
def read_filename(dir): 
    file_list = os.listdir(dir)
    return file_list

#전체 경로에서 파이명만 추출
def extract_filename(full_name): 
    file_name = os.path.basename(full_name)
    return file_name


#################################################
# df의 헤더 및 제목정보 출력
#################################################
def df_info(str, df):
    df_header = list(df.columns) #컬럼정보
    print('[dbg]', str, '헤더 : ', df_header[:8], '...')
    print('[dbg]', str, 'shape : ', df.shape[0], '행', df.shape[1], '열')
    
#################################################
# df의 행, 열 정보 리턴
#################################################
def get_df_shape(df):
    return df.shape[0], df.shape[1]
    

#################################################
# 숫자 셀중 , " 등이 들어간 경우가 있기 때문에 숫자값을 정규화
#################################################
def norm_value(var, remove_char):
    result = var
    
    if isinstance(result, str): #문자열만 작업 수행
        for ch in remove_char:
            result = result.replace(ch, '')
        
    return result


#################################################
# 템플릿데이터와 입력된 데이터 검증
# return doc_title : 해당 문서명을 기준으로 테이블 생성여부 확인
#################################################
def compare_schema(template, excel, sheet_name): 
    _dbg_('템플릿 파일 : ', template, sheet_name)
    _dbg_('엑셀 파일 : ', excel, sheet_name)
    # excel file load
    #%pip install openpyxl
    df_template = pd.read_excel(template, sheet_name = sheet_name) #템플릿과 일치여부를 확인하자
    df_excel = pd.read_excel(excel, sheet_name = sheet_name)

    # 데이터프레임의 nan 값을 None으로 변경
    #df.dropna(inplace=True)
    #df = df.where((pd.notnull(df)), 'None')
    df_template = df_template.fillna('None')
    df_excel = df_excel.fillna('None')
    
    
    ###################################
    # 첫줄 컬럼 헤더 비교
    ###################################
    df_template_header = list(df_template.columns) #템플릿 헤더
    df_excel_header = list(df_excel.columns) #df 헤더
    
    #_dbg_(df_excel_header[0]) #['2023년_ [4-다. 신입생 충원 현황_]        doc_title :  7 23 2023년_ [4-다. 신입생 충원 현황_]             doc_title :  11 22 2023년_ [4-다. 신입생 충원 현황_] +1 -1
    doc_title = df_excel_header[0]
    pos_1 = doc_title.find('. ')
    pos_2 = doc_title.find('_]')
    
    if pos_1 <= 0 or pos_2 <= 0:
        _dbg_('[종료] 엑셀 문서 제목 확인 필요 ', df_excel_header[0])
        sys.exit(0)
        
    doc_title = doc_title[pos_1+2:pos_2]
    #_dbg_('doc_title : ', pos_1, pos_2, df_excel_header[0], '[' + doc_title + ']') #['2023년_ [4-다. 신입생 충원 현황_]             doc_title :  11 22 2023년_ [4-다. 신입생 충원 현황_] [ 신입생 충원 현황]
    _dbg_('문서 제목 : ', df_excel_header[0], '[' + doc_title + ']') #['2023년_ [4-다. 신입생 충원 현황_]             doc_title :  11 22 2023년_ [4-다. 신입생 충원 현황_] [ 신입생 충원 현황]
    
    df_template_header[0].find('[', 0)
    df_excel_header[0].find('[', 0)

    df_template_header[0] = '' #첫행 첫열은 0000년_ [2-가-1. 전공과목 성적 분포_]
    df_excel_header[0] = ''
    
    _dbg_('df_template_header', df_template_header)
    _dbg_('df_excel_header', df_excel_header)
    if df_template_header != df_excel_header:
        _dbg_('[종료] : 첫줄 데이터 구조가 다름(1행)')               ############ ryu, 수작업 추가 : 2022년_ [12-라-2. 외국대학과 학점 교류 현황_] 과 2023년_ [12-라-2. 외국대학과 학점 교류 현황_]의 헤더가 다름
        _dbg_('df_template_header : '. df_template_header[0])
        _dbg_('df_excel_header : '. df_excel_header[0])
        sys.exit(0)

    #_dbg_('df_template', type(df_template), df_template)
    #_dbg_('df_template.shape[0]', type(df_template.shape[0]), df_template.shape[0])


    ###################################
    # 2행 이하의 제목행 비교
    ###################################
    df_template_shape0 = df_template.shape[0]
    for n in range(df_template_shape0): #엑셀의 첫 4행정도만 확인
        template_rows_list = list(df_template.loc[n]) #컬럼 시리즈를 리스트로 변경
        excel_rows_list = list(df_excel.loc[n]) #컬럼 시리즈를 리스트로 변경
        
        template_rows_list_mod = [word for word in template_rows_list if "\n" not in word]
        excel_rows_list_mod = [word for word in excel_rows_list if "\n" not in word]
        
        #_dbg_(n, 'template_rows_list', template_rows_list)
        #_dbg_(n, 'excel_rows_list', excel_rows_list)


        if template_rows_list_mod != excel_rows_list_mod:
            _dbg_('[종료] : {}번째 줄 데이터 구조가 다름(1행)'.format(n))
            _dbg_('template_rows_list : ', template_rows_list_mod)
            _dbg_('excel_rows_list : ', excel_rows_list_mod)            
            sys.exit(0)
            #return 0, 0, 0
    
    _dbg_('성공 : 두개의 작업문서 구조 일치함')
    #_dbg_('문서 헤더 - df_template_header : ', df_template_header)
    #_dbg_('문서 shape : ', df_template.shape[0], '행', df_template.shape[1], '열')
    #df_info('df_template', df_template)
    return df_excel, doc_title, df_template_shape0


#################################################
# df_excel에서 2~3번째 빈행 삭제 후 헤더재설정
#################################################
def remove_empty_row1(df_excel, df_template_shape0=0):
    _dbg_('df_template_shape0 : ', df_template_shape0)

    #_dbg_('df_excel.info() : ', type(df_excel.info()), df_excel.info())
    
    #현재 헤더 정보 확인
    header_info = df_excel.columns.tolist()
    #_dbg_('df_excel의 헤더 : ', header_info)
    df_info('원본 df_excel 헤더', df_excel)
    
    
    excel_rows_list = []
    
    new_header = []
    new_header_cnt = 0
    
    ## 학교 > 학교명 등 특정잘못된 제목부터 변경
    # 한번 추가한 헤더 컬럼을 또 추가하지 않기 위해 플랙 추가... 일부 학교명과 학교가 중보되어 들어간 문서도 있다. 2023년 _대학_14-마-2-(1). 기숙사 운영 결과(국·공립_민자)_학교별자료.xlsx
    flag_year = 0
    flag_utype = 0
    flag_estblish = 0
    flag_local = 0
    flag_status = 0
    flag_uname = 0
    
    for n in range(df_template_shape0): #템플릿 엑셀기준 행 전체정보 확인
        excel_rows_list = list(df_excel.loc[n]) #컬럼 시리즈를 리스트로 변경
        _dbg_(n, '행 데이터 : ', excel_rows_list[:8], "...")
        for m in range(len(excel_rows_list)): #각 열 순회
            cell = excel_rows_list[m]
            cell = cell.replace("\n", "")
            #_dbg_('cell 값 : ', cell)
            if cell == '기준연도' or  cell == '연도' and flag_year == 0:
                new_header.append('기준연도')
                new_header_cnt += 1
                flag_year = 1
            elif cell == '학교종류' and flag_utype == 0:
                new_header.append('학교종류')
                new_header_cnt += 1
                flag_utype = 1
            elif cell == '설립구분' and flag_estblish == 0:
                new_header.append('설립구분')
                new_header_cnt += 1
                flag_estblish = 1
            elif cell == '지역' and flag_local == 0:
                new_header.append('지역')
                new_header_cnt += 1
                flag_local = 1
            elif cell == '상태' or cell == '상태정보' and flag_status == 0:
                new_header.append('상태')
                new_header_cnt += 1
                flag_status = 1
            elif cell == '학교명' or cell == '학교' and flag_uname == 0:
                new_header.append('학교명')
                new_header_cnt += 1
                flag_uname = 1
                
    #나머지 new_header 정의
    for m in range(len(excel_rows_list)-new_header_cnt): #각 열 순회
        hdr = 'new-hdr-{}'.format(m)
        new_header.append(hdr)

    #새로운 헤더 지정
    if len(new_header) <= 0:
        _dbg_('오류 : 새로운 헤더를 생성하지 못함 : ', new_header)
        sys.exit(0)
        
    df_excel.columns = new_header
    _dbg_('새로운 헤더를 지정함 : ', new_header[:8])

    #기준연도 열 정보 읽기
    find_year = ''
    year_col = df_excel['기준연도'].to_list()
    #_dbg_('year_col', type(year_col), year_col)

    for str in year_col:
        if str != '기준연도' and str != 'None' and str != 'none' and str != '':
            find_year = str
            

    if find_year == '':
        _dbg_('오류 : 연도를 찾을 수 없음 ', find_year)
        sys.exit(0)
    else:
        _dbg_('성공 : 발견한 기준년도 = ', find_year)


    #나머지 제목행들 삭제
    #일부 2023년 _대학_4-가. 입학전형 유형별 선발 결과_학교별자료.xlsx 처럼 엑셀제목행은 6행으로 병합된었지만 실제는 5행인 경우가 있음, 
    #따라서 아래 몇행정도를 더 확인 후 None들로만 채워진 행 정보를 확인해서 삭제 해야함
    search_more_row = 5
    for n in range(df_template_shape0+search_more_row):
        excel_rows_list = list(df_excel.loc[n]) #컬럼 시리즈를 리스트로 변경

        if n < df_template_shape0:
            _dbg_('{}행 - 삭제 : {}, "..."'.format(n, excel_rows_list[:18]))
            df_excel = df_excel.drop(n) #빈행 삭제
        else: #삭제 후 엑셀제목행 5개정도 아래의 None값들의 수를 확인해보자    
            #_dbg_('excel_rows_list : ', excel_rows_list)
            none_cnt = excel_rows_list.count('None')
            row_len = len(excel_rows_list)
            if none_cnt == row_len: # None행임
                _dbg_('{}행 - 삭제 : {}, "..."'.format(n, excel_rows_list[:18]))
                df_excel = df_excel.drop(n) #빈행 삭제
                
    

    df_excel = reset_index(df_excel)

    ## 삭제 후 정리된 헤터 정보 출력
    df_info('( remove_empty_row1 ) 정리된 df_excel ', df_excel)
    

    # ## 삭제 후 정리된 행정보 출력
    # for n in range(df_template_shape0): #템플릿 엑셀기준 행정보 확인
    #     #_dbg_(df_template_shape0, n)
    #     excel_rows_list = list(df_excel.loc[n]) #컬럼 시리즈를 리스트로 변경
    #     _dbg_(n, excel_rows_list)





# header_mapping = {
#     'old_header1': 'new_header1',
#     'old_header2': 'new_header2',
#     # Add more mappings as needed
# }

# # Rename the headers using the dictionary
# df_excel.rename(columns=header_mapping, inplace=True)





    return df_excel, find_year

#################################################
# df 헤더명이 다른 경우가 있음 학교명 vs 학교...
#################################################
#def refine_header(df_excel):
    # _dbg_()

    # df_excel_header = list(df_excel.columns) #df 헤더

    # #엑셀의 [1,1]은 2023년_ [4-자. 신입생의 출신 고등학교 유형별 현황_]
    # df_excel_header[0].find('[', 0)
    # df_excel_header[0] = '기준연도'
    
    # for n in range(len(df_excel_header)):
    #     if df_excel_header[n] == '학교':
    #         df_excel_header[n] = '학교명'
            
    # _dbg_('df_excel_header', df_excel_header)
    
    # #타이틀 컬럼명 재지정
    # df_excel.columns = list(df_excel_header)
    
    #return df_excel


#################################################
# df_excel에서 2~3번째 빈행 삭제 후 헤더재설정
#################################################
def remove_empty_row2(df_excel):

    index_values = list(df_excel.index.values) #전체 행 인덱스 정보
    
    for idx in index_values:
        cell_year = df_excel.loc[idx, '기준연도']
        cell_univ = df_excel.loc[idx, '학교명']
        #print('remove_empty_row2 (cell_univ) : ', cell_univ)
        cell_type = df_excel.loc[idx, '학교종류']
        
        if cell_year == '기준연도' and cell_univ == '학교명' and cell_type == '학교종류':
            _dbg_('{}행 삭제'.format(idx), cell_year, cell_univ, cell_type)
            df_excel = df_excel.drop(idx) #제목행 복사로 반복행 제거
        elif cell_year == '기준연도' and cell_univ == '학교명' or cell_univ == '학교':
            _dbg_('{}행 삭제'.format(idx), cell_year, cell_univ)
            df_excel = df_excel.drop(idx) #제목행 복사로 반복행 제거   
        
                     

        if cell_year == 'None' and cell_univ == 'None' and cell_type == 'None':
            _dbg_('{}행 삭제'.format(idx), cell_year, cell_univ, cell_type)
            df_excel = df_excel.drop(idx) #제목 3행중 2행 삭제
        #elif cell_year != 'None':
            #find_year = cell_year
            #_dbg_('찾은 년도 : {}'.format(find_year))
    
    df_excel = reset_index(df_excel)
    #print('찾은 년도 : {}'.format(find_year))
    
    
    header_info = df_excel.columns.tolist()
    _dbg_('header_info : ', type(header_info), header_info[:8], '...')

    
    return df_excel#, find_year
        

#################################################
# df_excel에서 [학교종류 | 설립구분 | 지역 | 상태] 열 삭제
#################################################
def remove_univ_col(df_excel):
    #_dbg_()
    
    # 동일패턴A remove_univ_col
    df_header = list(df_excel.columns) #컬럼정보
    
    for hdr in df_header:
        if hdr == '설립구분':
            df_excel = df_excel.drop(['설립구분'], axis = 1)
        elif hdr == '학교종류':
            df_excel = df_excel.drop(['학교종류'], axis = 1)
        elif hdr == '지역':
            df_excel = df_excel.drop(['지역'], axis = 1)
        elif hdr == '상태':
            df_excel = df_excel.drop(['상태'], axis = 1)
        elif hdr == '상태정보':
            df_excel = df_excel.drop(['상태정보'], axis = 1)

    ## 삭제 후 정리된 헤터 정보 출력
    df_info('( remove_univ_col ) df_excel 컬럼 삭제 완료 - ', df_excel)

    return df_excel
            
#################################################
# 연도 찾아서 채우기
#################################################
def fill_year_col(df_excel, find_year, col_name = '기준연도',):
    #_dbg_(df_excel.head())
    #_dbg_(find_year, col_name)
    
    #column_values = list(df_excel[col_name])
    #print(column_values)
        
    #df_excel = df_excel[col_name].fillna(cell, inplace=True)
    df_excel[col_name].replace('None', find_year, inplace=True)
    
    _dbg_('성공 : ', find_year, '년도 채우기')
    #_dbg_(df_excel.head(5))
    return df_excel


#################################################
# 삭제된 df index 재설정
#################################################
def reset_index(df):
    #삭제된 줄만큼 루프를 더 돌던지.... 인덱스 값을 재조정하던지
    df = df.reset_index(drop=True) #(inplace=True)
    return df
       

#################################################
# 병합된 셀들의 none값을 채우기
#################################################
def fill_none_col(df_excel, col_name = '학교명'):
    
    col_header = list(df_excel.columns) #헤더부분 컬럼정보
    _dbg_('col_header', col_header)
    row_idx = list(df_excel.index.values) #전체 행 수
    col_data = list(df_excel[col_name]) #헤더명에 일치하는 컬럼 데이터
    _dbg_('col_data', col_data)
    
    cur_cell = ''
    
    idx_max = len(row_idx)-1

    for idx in row_idx:
        #print(idx, idx_max)
        if idx >= idx_max:
            break
        
        cur_cell = df_excel.loc[idx, col_name]
        next_cell = df_excel.loc[idx+1, col_name]
        
        # if col_name == '기준연도':
        #     _dbg_(idx, cur_cell, next_cell)
        
        # None으로 된 셀들에 대해서만 작업.... 따라서 숫자 셀들은 영향 없음음
        #a.empty, a.bool(), a.item(), a.any() or a.all().
        
        if cur_cell != '' and next_cell == 'None':
            #_dbg_('[{}] 병합셀 입력({}) : cur_cell = {} , next_cell = {} '.format(col_name, cur_cell, cur_cell, next_cell))
            df_excel.loc[idx+1, col_name] = cur_cell

    return df_excel



#################################################
# 병합된 셀들 데이터 채우기
#################################################
def fill_merge_cell(df_excel):
    
    col_header = list(df_excel.columns) #전체 헤더 컬럼정보
    row_idx = list(df_excel.index.values) #전체 행 수
    #col_data = list(df_excel[col_name]) #헤더명에 일치하는 컬럼 데이터
    

    for col_cnt in range(len(col_header)):
        col_name = col_header[col_cnt]
        
        df_excel = fill_none_col(df_excel, col_name=col_name) # 'new-hdr-0'
        
        #col_data = list(df_excel[col_name]) #헤더명에 일치하는 컬럼 데이터
        _#dbg_(col_name, col_data)
        
    _dbg_('병합된 셀들에 None값 대체하기 완료')
    _dbg_(df_excel.head())
            
    return df_excel


#################################################
# 대학명에서 좌우공백 제거... 전체 공백을 지우는게 유리할수도...
#################################################
def strip_univ_name(df_excel, col_name='학교명'):
    
    for n in range(len(df_excel)): #0~491 데이터프레임에서 학교명을 코드명으로 변경
        df_excel_univname = str(df_excel.loc[n, col_name])
        
        #df_excel_univname = df_excel_univname.strip() 전체
        df_excel_univname = df_excel_univname.lstrip()
        df_excel_univname = df_excel_univname.rstrip()

    _dbg_('학교명 공백 제거 완료')
    return df_excel



#################################################
# university테이블에서 특정 또는 가장 큰 코드값 읽기
#################################################
def get_univ_maxcode(curs):
    _dbg_()
    
    sql = 'SELECT {} FROM {}'.format('univ_cd', 'university') #university 테이블에서 가장큰 univ_code값을 가져오기 위함
    curs.execute(sql)
    code = curs.fetchall() #university에 등록된 대학교 목록
    
    max_code_val = 0
    
    for cd in code:
        #print('학교코드', type(cd), cd, cd['univ_cd'])
        
        code_str = cd['univ_cd']
        if code_str.find('-') == -1: #특수 기호 없다면
            max_code_val = max(max_code_val, int(code_str))
            
    #기존 테이블에 최대값은 10000x으로 되기 때문에 문자열로해도 100000x
    max_code_val += 1
    return str(max_code_val)
    
#################################################
# 유틸 : 문자열내 공백 제거, 문자열 교체
#################################################
def refine_univ_name(str):
    # A단계에서 처리가 안되는경우 2차 예외처리
    # 강릉원주대학교 vs 국립강릉원주대학교 와 같은 식으로 프리픽스와 포스트가 다른 경우가 있다.
    # 하나의 코드명으로 처리해야하기 때문에 유사한 대학명을 동일하게 매핑해야함            
    # 1. 국립 : 국립강릉원주대학교, 경상국립대학교와 같은 패턴, # 2. 숫자로 된 캠퍼스는 숫자만 일치해도 동일하게 처리

    str = str.lstrip()
    str = str.rstrip()
    str = str.replace('국립', '') #국립제거
    str = str.replace(' ', '') #띄어쓰기제거
    return str


#################################################
# 데이터 프레임의 학교명을 코드명으로 변경
# - 학교명과 일치하는 univ_cd를 찾아서 데이터프레임 정리
# conn, dbname, excel_file, sheet_name : university 테이블에 등록되지 않은 학교를 발견한 경우 다시 정보를 읽어오기 위함
# curs와 df_excel 만 실제로 필요함
#################################################
def set_univ_code(curs, conn, df_excel, dbname, template_file, excel_file, sheet_name):
    _#dbg_(excel_file, sheet_name)
    
    _dbg_('학교명을 코드명으로 변경 시작')

    add_new_univ_code = False #동일 대학명이 여러개 들어갈 수도 있기 때문에 DB에 등록된 정보를 다시 읽어와야 함

    sql_select = 'SELECT {}, {}, {} FROM {}'.format('univ_cd', 'univ_name_arlimi', 'univ_name', 'university') #university 테이블 기준으로 매핑하기 때문에 고정 값
    curs.execute(sql_select)

    univ_data = curs.fetchall() #university에 등록된 대학교 목록
    #_dbg_(type(univ_data), univ_data) # <class 'list'> [{'univ_cd': '0000001', 'univ_name_arlimi': '국립강릉원주대학교', 'univ_name': '강릉원주대학교1(강릉)'}, {'univ_cd': '0000002', 'univ_name_arlimi': '국립강릉원주대학교 _제2캠퍼스', 'univ_name': '강릉원주대학교2(원주)'},  ...
    
    # print(type(univ_data), univ_data)
    # print(type(univ_data[0]), univ_data[0])
    # print(type(univ_data[0]['univ_name_arlimi']), univ_data[0]['univ_name_arlimi'], univ_data[0]['univ_cd'])
    #conn.commit()

    #print('len(df_excel) : ' , len(df_excel))
    for n in range(len(df_excel)): #엑셀 데이터프레임에서 학교명을 추출

        df_excel_univname = str(df_excel.loc[n, '학교명'])
        #_dbg_('refine 전 : ', df_excel_univname)
        df_excel_univname = refine_univ_name(df_excel_univname)
        #_dbg_('refine 후 : ', df_excel_univname)
                    
        find_univ_code = False #university 테이블에 대학코드가 등록되지 않는 경우도 있음
        
        # A.엑셀대학명과 university 테이블의 코드명 매핑 - 대부분은 여기서 완료됨
        for i in range(len(univ_data)): #466 university 테이블에서 검색
            #if df_excel_univname == '화성의과학대학교':
            #print(n, df_excel_univname)

            univ_name_arlimi = univ_data[i]['univ_name_arlimi']
            univ_name_arlimi = refine_univ_name(univ_name_arlimi)
            
            # 일단은 사용하지 않음
            # univ_name_general = univ_data[i]['univ_name']
            # univ_name_general = refine_univ_name(univ_name_general)
            
            

            #(알리미 또는 univ_name명칭) 엑셀 학교명과 university 테이블에서 일치하는 학교명 발견하면 코드로 대체
            if df_excel_univname == univ_name_arlimi:
                #dat = '알리미 명칭 = df_excel_univname : {}, univ_name_arlimi : {}, univ_name_general : {}'.format(df_excel_univname, univ_name_arlimi, univ_name_general)
                #_dbg_(dat)
                #_dbg_('학교명 :', df_excel.loc[n, '학교명'], '학교코드 : ', univ_data[i]['univ_cd'])
                df_excel.loc[n, '학교명'] = univ_data[i]['univ_cd']
                find_univ_code = True
                break
            # elif df_excel_univname == univ_name_general:
            #     dat = '일반 명칭 = df_excel_univname : {}, univ_name_arlimi : {}, univ_name_general : {}'.format(df_excel_univname, univ_name_arlimi, univ_name_general)
            #     _dbg_(dat)
            #     _dbg_('학교명 :', df_excel.loc[n, '학교명'], '학교코드 : ', univ_data[i]['univ_cd'])
            #     df_excel.loc[n, '학교명'] = univ_data[i]['univ_cd']                
            #     find_univ_code = True
            #     break

        
        # 아래 부분은 재작업하자... 24.02.16
        # universiti 테이블에 학교가 등록되지 않음 경우 -------------------어쩌다 한번씩 벌어질 일이기 때문에 그냥 재작업하자    경남정보..[폐교] 같은 케이스
        # 코드가 없는 학교명을 찾음
        # df_exce에서 없는 정보를 읽어옴
        # 신규 학교를 university 테이블에 등록
        # 신규 코드 부여
        if find_univ_code == False: #university 테이블에 등록되지 않은 학교를 발견, DB에 등록, ex)신경주대학교
            _dbg_('\n\n학교명과 일치하는 코드를 찾지 못함(신규 학교를 등록해야함) ', n, '번째 인덱스', df_excel_univname)
            
            
            #df_excel, doc_title, df_template_shape0 = compare_schema(template_file, excel_file, sheet_name) #get_table_name(doc_title) 에서 사용
            #df_excel, find_year = remove_empty_row1(df_excel, df_template_shape0)
            
            df_excel_add = pd.read_excel(excel_file, sheet_name = sheet_name)
            df_template_add = pd.read_excel(template_file, sheet_name = sheet_name)
            col_template, row_template = get_df_shape(df_template_add)
            #_dbg_('col_template', type(col_template), col_template)
            #_dbg_('row_template', type(row_template), row_template)
            
            df_excel_add = df_excel_add.fillna('None')
            df_excel_add, find_year = remove_empty_row1(df_excel_add, col_template) #
            df_excel_add = remove_empty_row2(df_excel_add)
            
            index_values = list(df_excel_add.index.values)
            for idx in index_values:
                #cell_year = df_excel.loc[idx, '기준연도']
                cell_univ_name = df_excel_add.loc[idx, '학교명']
                cell_univ_name = refine_univ_name(cell_univ_name)
                
                #_dbg_(idx, cell_univ_name)
                
                #신규 대학 발견, 관련 정보를 university 테이블에 등록
                if cell_univ_name == df_excel_univname:
                    #_dbg_(df_excel_add.head())
                    #_dbg_(idx)
                    _dbg_('신규 대학 발견, university 테이블에 등록해야 함 (행번호, 다시 오픈한 엑셀 대학명, 엑셀대학명)', idx, cell_univ_name, df_excel_univname)
                    
                    # 동일패턴A remove_univ_col
                    df_header = list(df_excel.columns) #컬럼정보
                    cell_estblish = ''
                    cell_type = ''
                    cell_locale = ''
                    cell_status = ''
                    for hdr in df_header:
                        if hdr == '설립구분':
                            cell_estblish = df_excel_add.loc[idx, '설립구분']
                            _dbg_('cell_estblish : ', cell_estblish)
                        elif hdr == '학교종류':
                            cell_type = df_excel_add.loc[idx, '학교종류']
                            _dbg_('cell_type : ', cell_type)
                        elif hdr == '지역':
                            cell_locale = df_excel_add.loc[idx, '지역']
                            _dbg_('cell_locale : ', cell_locale)
                        elif hdr == '상태':
                            cell_status = df_excel_add.loc[idx, '상태']
                            _dbg_('cell_status : ', cell_status)
                        elif hdr == '상태정보':
                            cell_status = df_excel_add.loc[idx, '상태정보']
                            _dbg_('cell_status : ', cell_status)
            
                    
                    #university 테이블의 코드값중 최대값을 찾음
                    max_univ_code = get_univ_maxcode(curs)
                    
                    _dbg_('등록할 코드 [{}] : 학교종류 ({}), 설립구분 ({}), 상태 ({}), 지역 ({})'.format(max_univ_code, cell_type, cell_estblish, cell_status, cell_locale))

                    #'insert into cosshubtest.university(univ_cd, univ_name_arlimi, univ_name, univ_name_en, univ_type, establish, status, campus, campus_name, locale, homepage, tel, fax) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
                    
                    col_name_str = '`univ_cd`, `univ_name_arlimi`, `univ_name`, `univ_name_en`, `univ_type`, `establish`, `status`, `campus`, `campus_name`, `locale`, `homepage`, `tel`, `fax`'
                    field_data_str = "'{}', '{}', '{}', 'None', '{}', '{}', '{}', 'None', 'None', '{}', 'None', 'None', 'None'".format(max_univ_code, cell_univ_name, cell_univ_name, cell_type, cell_estblish, cell_status, cell_locale)
                    sql_new_univ = 'insert into {}.{}({}) values({})'.format(dbname, 'university', col_name_str, field_data_str)
                    
                    curs.execute(sql_new_univ)
                    conn.commit()
                    
                    #동일 대학명이 여러개 들어갈 수도 있기 때문에 DB에 등록된 정보를 다시 읽어와야 함
                    sql_select = 'SELECT {}, {} FROM {}'.format('univ_cd', 'univ_name_arlimi', 'university') #university 테이블 기준으로 매핑하기 때문에 고정 값
                    _dbg_(sql_select)
                    curs.execute(sql_select)
                    univ_data = curs.fetchall() #university에 등록된 대학교 목록
                    
                    
                    #엑셀 학교명과 university 테이블에서 일치하는 학교명 발견하면 코드로 대체
                    #if df_excel_univname == univ_data[i]['univ_name_arlimi']:
                        #print(univ_data[i]['univ_name_arlimi'], univ_data[i]['univ_cd'])
                        
                    df_excel.loc[n, '학교명'] = max_univ_code
            
    return df_excel



#################################################
# 기준이 되는 학교정보 테이블 입력, 외부데이터와는 달리 자체 생성 데이터
#################################################
def create_university(conn, curs, dbname, table, excel_path, sheet_name, table_reset = False, mode_sql = 'I/U/D'):
    _dbg_(table)
    df = pd.read_excel(excel_path, sheet_name = sheet_name)

    # 데이터프레임의 nan 값을 None으로 변경
    #df.dropna(inplace=True)
    #df = df.where((pd.notnull(df)), 'None')
    df = df.fillna('None')

    #df.rename(columns={'대학 고유코드':'a'}, inplace=True) #특정 컬럼명 변경
    #df.columns('a', 'b') 

    cols_ko = list(df.columns) #컬럼 시리즈를 리스트로 변경
    cols_en = list(df.iloc[0]) #데이터 시리즈를 리스트로 변경

    for n in range(len(cols_ko)):
        #print(cols_en[n], cols_ko[n])
        df.rename(columns={cols_ko[n]:cols_en[n]}, inplace=True) #특정 컬럼명 변경
    df = df.drop(0) #특정행 삭제
    
    # db truncate tables, 기존 데이터를 삭제 후 인서트
    if table_reset == True:
        sql_truncate = 'truncate table {}.{}'.format(dbname, table)
        curs.execute(sql_truncate)
        conn.commit()
        print('{}.{} 테이블 삭제 완료'.format(dbname, table), sql_truncate)
    
    # data insert to db
    field_names = ''
    field_data = ''


    for n in range(len(cols_en)):
        if n == 0 :
            field_names = cols_en[n]
            field_data = '%s'
        else:
            field_names = field_names + ', ' + cols_en[n]
            field_data = field_data + ', ' + '%s'
            
        #입력데이터 중 좌우측 공백제거
        field_data = field_data.lstrip()
        field_data = field_data.rstrip()
        
        
    # sql_insert = 'insert into cosshubtest.university(univ_cd, univ_name_arlimi, univ_name, univ_name_en, univ_type, establish, status, campus, campus_name, locale, homepage, tel, fax) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
    # print(sql_insert)

    univ_name_arlimi_date = ''
    univ_cd=''
    if mode_sql == 'I': #insert mode
        sql = 'insert into {}.{}({}) values({})'.format(dbname, table, field_names, field_data)
    elif mode_sql == 'U': #update mode, 알리미 대학명 이름만 변경함 기준은 대학코드 >>>> 변경된 데이터만 찾아서 향후 별도 처리
        sql = 'update {}.{} set {}={} where univ_cd={}'.format(dbname, table, 'univ_name_arlimi', univ_name_arlimi_date, univ_cd)
    
    print('1번 : ', sql)
    
    for idx in range(len(df)):
        #print(idx, sql)
        curs.execute(sql, tuple(df.values[idx]))
    conn.commit()
    
    if mode_sql == 'I': #insert mode
        print(idx, '개의 데이터 입력 성공')
    elif mode_sql == 'U': #update mode, 알리미 대학명 이름만 변경함 기준은 대학코드
        print(idx, '개의 데이터 업데이트 성공')
    
    # UPDATE cosshubtest.university
	# SET univ_name_arlimi='None-'
	# WHERE univ_cd='0002653'
 
 
 


In [4]:
%load_ext autoreload
%autoreload 2


In [5]:
import sys, os

#file = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료'
#sys.path.append('D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료')

#from tables_schema import *
#from tables_schema import tables_schema
#import tables_schema

#################################################
# 테이블의 존재 여부를 확인
#################################################
def chk_table(curs, dbname, table):
    _dbg_(table)
    sql = "select 1 from INFORMATION_SCHEMA.TABLES where TABLE_SCHEMA = '{}' AND TABLE_NAME = '{}'".format(dbname, table)
    #print('chk_table : ', sql)
    ret = curs.execute(sql)
    
    if ret != 1:
        _dbg_('{}.{} 테이블이 없으므로 생성해야함'.format(dbname, table))
    else:
        _dbg_('{}.{} 테이블이 존재함'.format(dbname, table))

    return ret

#################################################
# 문서명에 해당하는 테이블 명을 확인, 띄어쓰기 모두 포함된 상태
#################################################
def get_table_name(doc_title):
    _dbg_('[{}]'.format(doc_title))
    _dbg_(sch.table_dic.get(doc_title))
    
    if sch.table_dic.get(doc_title) == None:
        _dbg_('[종료] [{}]를 저장할 테이블 구조가 없으므로 [table_dic]에 구조부터 생성해야함'.format(doc_title))
        sys.exit(0)
    else:
        table_name = sch.table_dic[doc_title]
        _dbg_('[{}]를 저장할 [{}] 테이블 존재함'.format(doc_title, table_name))
    
    return table_name


#################################################
# 테이블 생성
#################################################
def create_table(curs, dbname, table):
    _dbg_(table)
    ret_val = ''
    ret_mode = '' #idx 가 있냐 없냐...

    sql = sch.table_dic[table]
    chk_tbl = chk_table(curs, dbname, table)
    if chk_tbl != 1: #테이블명에 해당하는 테이블이 없음
        _dbg_('create_table : ', sql)
        rs = curs.execute(sql)
        
        if rs !=0 :
            _dbg_('[오류_종료]create_table 생성오류 [{}]: {}'.format(rs, table))
            os.exit(0)
        else:
            _dbg_('{}.{} 신규 테이블 생성 성공'.format(dbname, table))
            ret_val = 'create'
    elif chk_tbl == 1:
        _dbg_('{}.{} 테이블이 존재함'.format(dbname, table))
        ret_val = 'exist'
    else:
        _dbg_('[오류_종료] 알수없는 생성오류 [{}]: {}'.format(rs, table))
        
    #_dbg_(type(sql), sql)
    
    if sql.find('`idx` int auto_increment') > 0:
        _dbg_('idx mode')
        ret_mode = 'idx_on'
    else:
        _dbg_('기준년도+학교코드 mode')
        ret_mode = 'idx_off'
    #    테이블에 idx가 있는 경우는 다르게 처리
    #    테이블에 idx가 있다면 인서트_업데이트를 다르게..
        
        
    return ret_val, ret_mode
    
    
#################################################
# 대학코드 -> 대학명
#################################################
def get_univ_name(curs, univ_code):
    _dbg_(univ_code)
    sql = 'SELECT {} FROM {} WHERE univ_cd = {}'.format('univ_name', 'university', univ_code) #university 테이블 기준으로 매핑하기 때문에 고정 값
    curs.execute(sql)

    univ_name = curs.fetchall() #university에 등록된 대학교 목록

    return univ_name
        
        
#################################################
# 대학명 -> 대학코드
#################################################
def get_univ_name(curs, univ_name):
    _dbg_(univ_name)
    sql = 'SELECT {} FROM {} WHERE univ_name_arlimi = {}'.format('univ_cd', 'university', univ_name) #university 테이블 기준으로 매핑하기 때문에 고정 값
    curs.execute(sql)

    univ_code = curs.fetchall() #university에 등록된 대학교 목록

    return univ_code
    
#################################################
# DB 테이블의 컬럼명과 데이터타입만 추출
#################################################
def get_db_column_name(curs, dbname, table):
    _dbg_(table)
    #cnt_columns = len(df_excel.columns) #엑셀 df읠 필드명

    sql_col = 'SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = "{}"'.format(table)

    curs.execute(sql_col)
    #conn.commit()
    col_info = curs.fetchall() #컬럼명과 데이터타입 리스트 안에 딕셔너리로...
    
    _dbg_(table, '테이블의 컬럼 정보 : ', type(col_info), len(col_info), col_info[:8])

    idx_mode = 'off' #기준연도+학교코드 모드와 idx 모드로 나뉨
    col_name_list = list()
    col_name_str = ''
    col_cnt = len(col_info)
    
    for n in range(col_cnt):
        col = col_info[n]['COLUMN_NAME']
        #_dbg_('[{}]'.format(col))
        if col == 'idx':
            idx_mode = 'on'
        else:
            #col_name_list.append(col_name[n]['COLUMN_NAME'])
            col_name_list.append(col)
            #_dbg_('appended : [{}]'.format(col))

    if idx_mode == 'on': #idx하나를 빼야 함
        col_cnt = col_cnt - 1

    for n in range(col_cnt):
        if n == 0:
            col_name_str = '`' + str(col_name_list[n]) + '`, '
        elif n >= col_cnt-1:
            col_name_str = col_name_str + '`' + str(col_name_list[n]) + '`'
        else:
            col_name_str = col_name_str + '`' + str(col_name_list[n]) + '`, '


    #print(col_name_str)
    # print(type(col_name), len(col_name), col_name)
    # print(type(col_name[0]), len(col_name[0]), col_name[0])
    # print(type(col_name[0]['COLUMN_NAME']), col_name[0]['COLUMN_NAME'])

    return col_info, col_name_str, col_name_list


#################################################
# 데이터 프레임의 의 컬럼명만 추출
#################################################
# def get_df_column_name(curs, dbname, table):




#################################################
# 테이블 필드명의 데이터 타입을 얻어옴
#################################################
def get_data_type(col_info, col_name):
    data_type = ''
    for info in col_info:
        #print(info['COLUMN_NAME'], info['DATA_TYPE'])
        if info['COLUMN_NAME'] == col_name:
            data_type = info['DATA_TYPE']
        
    return data_type


# {COLUMN_NAME': 'idx', 'DATA_TYPE': 'int'}
# {'COLUMN_NAME': 'year', 'DATA_TYPE': 'int'}
# {'COLUMN_NAME': 'univ_cd', 'DATA_TYPE': 'varchar'}



#################################################
# 기준년도 + 학교코드가 키 값이 아니라 idx기준을 데이터 인서트
#################################################
def insert_update_df_idx(curs, conn, df_excel, dbname, table, mode = 'create'):
    _dbg_(table, '에 데이터 입력 시작작')
    cnt_insert = 0
    cnt_update = 0

    #DB 테이블의 헤더정보만 추출    
    col_info, col_name_org, col_name_list = get_db_column_name(curs, dbname, table) #리턴값중 col_info만 사용

    # 테스트코드 데이터 타입만 읽어오기    
    # data_type = get_data_type(col_info, 'year')
    # _dbg_(data_type)
    
    #_dbg_('col_info : ', len(col_info), col_info)
    #_dbg_('col_name_list : ', len(col_name_list), col_name_list)
    
    
    
    # reg_date와 mod_date는 처리에서 제외
    col_info_cnt = len(col_info)
    for i in range(len(col_info)):
        if col_info[i]['COLUMN_NAME'] == 'reg_date' and col_info[i]['DATA_TYPE'] == 'datetime':
            col_info_cnt = col_info_cnt - 1
        elif col_info[i]['COLUMN_NAME'] == 'mod_date' and col_info[i]['DATA_TYPE'] == 'datetime':
            col_info_cnt = col_info_cnt - 1
            
    # print(df_excel.head(15))
    # sys.exit(1)

    #df_excel 데이터프레임에서 읽어온 데이터와 테이블에 있는 데이터를 비교해서 변경된 사항은 업데이트
    for n in range(len(df_excel)): #전처리 완료된 데이터 프레임 전체를 읽어서 DB에 업데이트

        field_data = list(df_excel.iloc[n]) # 시리즈 -> 리스트
        _dbg_('입력할 행 data : [', n, '] ', field_data)
        
        first_field = True #sql생성을 위해 처음과 중간 끝 작업을 위해
        idx_cnt = 0 #idx는 자동증가이기 때문에 처리하지 않기때문에 field_data[m]과 -1개 차이가 발생함
        col_name_str = ''
        col_data_str = ''
        select_where = ''
        update_data = ''
        
        
        for m in range(col_info_cnt):
            
            #idx는 자동증가이므로 제외
            col_name = col_info[m]['COLUMN_NAME']
            col_type = col_info[m]['DATA_TYPE']
            
            #_dbg_(col_name, col_type, field_data[m-idx_cnt])
            
            #_dbg_(col_name, col_type)
            if(col_name == 'idx'):
                _dbg_("idx mode ON")
                idx_cnt = 1
            else: #idx이외의 나머지 필드
                
                if first_field == True:
                    #_dbg_('처음({}/{}), col_name:{}, col_type:{}'.format(m, col_info_cnt, col_name, col_type))  #idx가 있으면 하나 빼야함!!
                    col_name_str = '`{}`, '.format(col_name) #처음
                    
                    _dbg_('첫열 데이터', col_name, col_type, field_data[m-idx_cnt])
                    if col_type == 'int':
                        #_dbg_('int', col_name, col_type, field_data[m-idx_cnt])
                        norm_val = norm_value(field_data[m-idx_cnt], ',') #, 제거(처음)
                        col_data_str = '{}, '.format(norm_val)
                        select_where = '`{}` = {} and '.format(col_name, norm_val)
                        
                    elif col_type == 'float':
                        #_dbg_('float', col_name, col_type, field_data[m-idx_cnt])
                        norm_val = norm_value(field_data[m-idx_cnt], ',') #, 제거(처음)
                        col_data_str = '{}, '.format(norm_val)
                        select_where = '`{}` = {} and '.format(col_name, norm_val)

                    elif col_type == 'bigint':
                        #_dbg_('bigint', col_name, col_type, field_data[m-idx_cnt])
                        norm_varchar = norm_value(field_data[m-idx_cnt], ',') #varchar 타입 . , 등은 허용(처음)
                        col_data_str = '{}, '.format(norm_val)
                        select_where = '`{}` = {} and '.format(col_name, norm_val) 
                        
                    elif col_type == 'varchar':
                        #_dbg_('varchar', col_name, col_type, field_data[m-idx_cnt])
                        norm_varchar = norm_value(field_data[m-idx_cnt], '"') #varchar 타입 . , 등은 허용(처음)
                        col_data_str = '"{}", '.format(norm_varchar)
                        select_where = '`{}` = "{}" and '.format(col_name, norm_varchar)
                    first_field = False
                    

                elif m >= col_info_cnt-1: #마지막
                    #_dbg_('끝({}/{}), col_name:{}, col_type:{}'.format(m, col_info_cnt, col_name, col_type))
                    col_name_str = col_name_str + '`{}` '.format(col_name) #마지막
                    
                    _dbg_('끝열 데이터', col_name, col_type, field_data[m-idx_cnt])
                    if col_type == 'int':
                        norm_val = norm_value(field_data[m-idx_cnt], ',') #, 제거(마지막)
                        col_data_str = col_data_str + '{}'.format(norm_val)
                        select_where = select_where + '`{}` = {}'.format(col_name, norm_val)
                        
                    elif col_type == 'float':
                        norm_val = norm_value(field_data[m-idx_cnt], ',') #, 제거(마지막)
                        col_data_str = col_data_str + '{}'.format(norm_val)
                        select_where = select_where + '`{}` = {}'.format(col_name, norm_val)

                    elif col_type == 'bigint':
                        #_dbg_('bigint', col_name, col_type, field_data[m-idx_cnt])
                        norm_varchar = norm_value(field_data[m-idx_cnt], ',') #varchar 타입 . , 등은 허용(처음)
                        col_data_str = col_data_str + '{}'.format(norm_val)
                        select_where = select_where + '`{}` = {}'.format(col_name, norm_val)
                        
                    elif col_type == 'varchar':
                        norm_varchar = norm_value(field_data[m-idx_cnt], '"') #varchar 타입 . , 등은 허용(마지막)
                        col_data_str = col_data_str + '"{}"'.format(norm_varchar)
                        select_where = select_where + '`{}` = "{}"'.format(col_name, norm_varchar)
                        
                    _dbg_(norm_val)

                else:
                    #_dbg_('중간({}/{}), col_name:{}, col_type:{}'.format(m, col_info_cnt, col_name, col_type))
                    col_name_str = col_name_str + '`{}`, '.format(col_name) #중간
                    norm_val = norm_value(field_data[m-idx_cnt], ',') #, 제거
                    
                    _dbg_('중간열 데이터', col_name, col_type, field_data[m-idx_cnt])
                    if col_type == 'int':
                        norm_val = norm_value(field_data[m-idx_cnt], ',') #, 제거(중간)
                        col_data_str = col_data_str + '{}, '.format(norm_val)
                        select_where = select_where + '`{}` = {} and '.format(col_name, norm_val)
                        
                    elif col_type == 'float':
                        norm_val = norm_value(field_data[m-idx_cnt], ',') #, 제거(중간)
                        col_data_str = col_data_str + '{}, '.format(norm_val)
                        select_where = select_where + '`{}` = {} and '.format(col_name, norm_val)
                        
                    elif col_type == 'bigint':
                        #_dbg_('bigint', col_name, col_type, field_data[m-idx_cnt])
                        norm_varchar = norm_value(field_data[m-idx_cnt], ',') #varchar 타입 . , 등은 허용(처음)
                        col_data_str = col_data_str + '{}, '.format(norm_val)
                        select_where = select_where + '`{}` = {} and '.format(col_name, norm_val)                
                        
                    elif col_type == 'varchar':
                        norm_varchar = norm_value(field_data[m-idx_cnt], '"') ##varchar 타입 . , 등은 허용(중간)
                        col_data_str = col_data_str + '"{}", '.format(norm_varchar)
                        select_where = select_where + '`{}` = "{}" and '.format(col_name, norm_varchar)

                                        
        _dbg_('정제 후 입력할 col_data_str : ', col_data_str)
        #_dbg_('입력할 norm_val : ', norm_val)

        #기존 테이블이 존재하면 업데이트 모드로 해야함            
        #테이블은 존재하지만 실제 데이터는 없을수도 있음
        if mode == 'exist':
            sql = 'select * from {}.{} where {}'.format(dbname, table, select_where) #table_dic['전공과목 성적 분포'] = 'major_grades' 만 해당
            #_dbg_(select_where)
            _dbg_('exist : ' , sql)
            #_dbg_(col_data_str)
            #_dbg_(field_data)
            
            cnt = curs.execute(sql) #검색된 갯수
            result = curs.fetchall() #리스트 안에 딕셔너리로...
            #print(type(cnt), cnt, type(result), result)

            ### Ryu, 업데이트시 키값만 정확하게 읽어오는 거 필요
            if cnt == 1: #정상 검색 -> 업데이트 대상    >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 키값 기준으로 검색을 해서 찾아야하는데 idx 모드일때 등 키값을 정확히 찾기가 어려움... 천천히 생각
                #print(cnt, '쿼리 검색 성공(업데이트) ')
                #if table == 'major_grades': #테이블로 구분하거나 semester로 구분해도 됨
                #sql = 'update {}.{} set {} where {}'.format(dbname, table, col_name_str, select_where)
                #_dbg_(cnt, '업데이트를 하면 좋은데 키값 기준에 대한 고민이 필요')
                #_dbg_(cnt, '기존 데이터 있음(업데이트 실행) : ', sql)
                # else:
                #     sql = 'update {}.{} set {} where year = {} and univ_cd = "{}"'.format(dbname, table, set_data, year, univ_cd)
                cnt_update += 1
            elif cnt == 0: #테이블은 존재하지만 데이터는 없는 경우이기 때문에 인서트를 해야함
                sql = 'insert into {}.{}({}) values({})'.format(dbname, table, col_name_str, col_data_str)
                _dbg_('업데이트 모드로 하는게 좋은데.... : ', sql)
                if sql != '':
                    curs.execute(sql)
                    
                cnt_insert += 1
            else:
                _dbg_(cnt, '오류 : 테이블이 존재하지만 알수 없는 오류 : ', sql)
                sys.exit(0)

        elif mode == 'create': 
            sql = 'insert into {}.{}({}) values({})'.format(dbname, table, col_name_str, col_data_str)
            _dbg_('신규 인서트 모드 : ', sql)
            cnt_insert += 1
            
            if sql != '':
                curs.execute(sql)
            #conn.commit()        
            
        else:
            _dbg_(cnt, '오류 : 정의되지 않음 ', sql)
            sys.exit(0)
        
    #     if sql != '':
    #         curs.execute(sql)
    conn.commit()
    
    print(table, '테이블 : ', cnt_insert, '개의 신규 인서트     ', cnt_update, '개의 업데이트')

In [6]:
##################################################################################################
# university 테이블, 단독작업
##################################################################################################

def create_university(dbname, curs, conn, reset_mode = False):
    excel_univ = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/05.설계/01.테이블설계/01.Codes.xlsx'
    sheet_name = '학교코드정보'

    table_name = 'university'

    df_univ = pd.read_excel(excel_univ, sheet_name = sheet_name)

    df_univ = df_univ.fillna('None')

    cols_ko = list(df_univ.columns) #컬럼 시리즈를 리스트로 변경
    cols_en = list(df_univ.iloc[0]) #데이터 시리즈를 리스트로 변경

    for n in range(len(cols_ko)):
        #print(cols_en[n], cols_ko[n])
        df_univ.rename(columns={cols_ko[n]:cols_en[n]}, inplace=True) #특정 컬럼명 변경
    df = df_univ.drop(0) #특정행 삭제


    # 기존 테이블을 삭제 후 인서트하는 경우
    if reset_mode == True:
        sql_truncate = 'truncate table {}.{}'.format(dbname, table_name)
        curs.execute(sql_truncate)
        conn.commit()

    ret, mode = create_table(curs, dbname, table = table_name)
    _dbg_('테이블 생성 정보 : ', ret, mode)

    if ret == 'create' or ret == 'exist': #create, exist
        #idx 모드와 무관하게 실행하자
        #insert_update_df_idx(curs, conn, df_excel, dbname, table = table_name) #df_excel을 table에 잘 비교해서 업데이트...
            
        # data insert to db
        field_names = ''
        field_data = ''

        for n in range(len(cols_en)):
            if n == 0 :
                field_names = cols_en[n]
                field_data = '%s'
            else:
                field_names = field_names + ', ' + cols_en[n]
                field_data = field_data + ', ' + '%s'
            
            
        # sql_insert = 'insert into cosshubtest.university(univ_cd, univ_name_arlimi, univ_name, univ_name_en, univ_type, establish, status, campus, campus_name, locale, homepage, tel, fax) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
        # print(sql_insert)

        sql_insert = 'insert into {}.{}({}) values({})'.format(dbname, table_name, field_names, field_data)
        #print(sql)
        for idx in range(len(df)):
            curs.execute(sql_insert, tuple(df.values[idx]))
        conn.commit()

        print("\n#########################################################################################################")
        print('##### university 테이블 생성 완료 : 테이블 = {}, 엑셀파일={}'.format(table_name,  excel_univ))
        print("#########################################################################################################\n")

In [7]:
##################################################################################################
# DB 및 작업 폴더 설정
##################################################################################################

# %load_ext autoreload
# %autoreload 2

dbname = 'swu_external'

#conn, curs = db_setting('133.186.247.101', 'root', 'coss!@#$', dbname = 'cosshubtest') #coss test
conn, curs = db_setting('13.124.114.176', 'root', 's!W@u#12', dbname = dbname)

sheet_name = 'Sheet1'
table = 'university'

# template_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/TEMPLATE/2.교육여건-Template'
# excel_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/EXCEL/2.교육여건/2.교육여건 - 2023년'

template_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/TEMPLATE/3.교육연구성과-Template'
excel_dir = 'D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/EXCEL/3.교육연구성과/3.교육연구성과 - 2023년'

template_file_list = read_filename(template_dir)
excel_file_list = read_filename(excel_dir)

# university 테이블 생성
# create_university(dbname, curs, conn, reset_mode = False)

In [8]:
excel_worked = [
    #{'doc_title' : '작업문서명', 'db_table' : '테이블명', 'excel_file' : '엑셀파일명'}, 
    #{'doc_title' : '작업문서명', 'db_table' : '테이블명', 'excel_file' : '엑셀파일명'}, 
]



for excel in excel_file_list:
    for template in template_file_list:
        if excel[4:] == template[4:]:
            print("\n\n")
            print("#########################################################################################################")
            print("##### 작업 엑셀 파일명 : ", excel)
            print("#########################################################################################################")

            template_file = '{}/{}'.format(template_dir, template)
            excel_file = '{}/{}'.format(excel_dir, excel)
            
            df_excel, doc_title, df_template_shape0 = compare_schema(template_file, excel_file, sheet_name) #get_table_name(doc_title) 에서 사용
            
            df_excel, find_year = remove_empty_row1(df_excel, df_template_shape0)

            #df_excel = remove_empty_row2(df_excel)

            df_excel = remove_univ_col(df_excel)
            #df_excel = fill_year_col(df_excel, find_year, col_name='기준연도')   # 2023상반기... 2023하반기와 같이 토글되는 경우들도 있음....
            df_excel = fill_merge_cell(df_excel) #학교명, 정원외 등 병합된 셀 통합
            
            # _dbg_(df_excel.head())
            # sys.exit(0)
            
            df_excel = strip_univ_name(df_excel, col_name='학교명') #대학명에서 좌우공백 제거
            df_excel = set_univ_code(curs, conn, df_excel, dbname, template_file, excel_file, sheet_name) #데이터프레임의 학교명을 코드명으로 변경, DB학교 공백처리...
            

            
            # 이제 데이터를 DB에 입력하면 됨
            table_name = get_table_name(doc_title)
            _dbg_('{}테이블에 {}데이터 입력 시작 : {}'.format(table_name, sch.table_dic.get(doc_title), doc_title))
            
            ret, mode = create_table(curs, dbname, table = table_name)
            if ret == 'create' or ret == 'exist': #create, exist
                #idx 모드와 무관하게 실행하자
                insert_update_df_idx(curs, conn, df_excel, dbname, table = table_name, mode = ret) #df_excel을 table에 잘 비교해서 업데이트...
            else:
                _dbg_('테이블 생성 오류 : {}'.format(table_name))
                sys.exit()

            print("\n#########################################################################################################")
            print('##### 작업 완료 : 테이블 = {}, 문서명={}, 엑셀파일={}'.format(table_name, doc_title, excel_file))
            print("#########################################################################################################\n")

            dict_item = {'doc_title' : '{}'.format(doc_title), 'db_table' : '{}'.format(table_name), 'excel_file' : '{}'.format(excel_file)}
            excel_worked.append(dict_item)


if len(excel_worked) <= 0:
    print("\n#########################################################################################################")
    print('##### 작업대상 파일이 없음 : {}개'.format(len(excel_worked)))
    print("#########################################################################################################\n")    
else:
    print("\n#######################################################################################################################################################################")
    print('##### 전체 작업 성공 : {}개'.format(len(excel_worked)))
    print("#######################################################################################################################################################################\n")

    for n in range(len(excel_worked)):
        print('\n----- ', n+1, '번째 작업파일 정보 ------')
        print('문서명 : ', excel_worked[n]['doc_title'])
        print('테이블 : ', excel_worked[n]['db_table'])
        print('파일명 : ', excel_worked[n]['excel_file'])
            
            





#########################################################################################################
##### 작업 엑셀 파일명 :  2023년 _대학_12-가. 연구비 수혜 실적_학교별자료.xlsx
#########################################################################################################
[dbg] ( compare_schema )  템플릿 파일 :  D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/TEMPLATE/3.교육연구성과-Template/0000년 _대학_12-가. 연구비 수혜 실적_학교별자료.xlsx Sheet1  
[dbg] ( compare_schema )  엑셀 파일 :  D:/OneDrive - MEDIOPIA TECH,Co.,Ltd/230801 서울여대/10.대학알리미 자료/EXCEL/3.교육연구성과/3.교육연구성과 - 2023년/2023년 _대학_12-가. 연구비 수혜 실적_학교별자료.xlsx Sheet1  
[dbg] ( compare_schema )  문서 제목 :  2023년_ [12-가. 연구비 수혜 실적_] [연구비 수혜 실적]  
[dbg] ( compare_schema )  df_template_header ['', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', '

SystemExit: 0

c:\Users\acery\anaconda3\envs\ml\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#print(sch.table_dic['dormitory_result_national_univ_capital'])